In [1]:
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
def triple_barrier_m(data, length, height_low, height_high):
    """
    Generate labels according to the 'Triple Barrier Method' and
    Calculates a metric balance of the labels,
    between 0 and 1, with 1 being the perfect balance.

    Args:
        data (dataframe):
        length (int): candles for vertical barrier.
        height (float): percentage for horizontal barriers
    Returns:
        datacopy (dataframe):
            Copy of dataframe input, adding label column.
        metrica_balance (float): between 0 and 1.
    """
    datacopy = data.copy()
    datacopy.drop(columns= "labels", inplace = True)
    output = list()
    high_barrier = list()
    low_barrier = list()
    vert_barrier = list()

    # I lose the last "length" data, so that it does not
    # calculate them with a decreasing candle window.
    for i in range(len(data)-length):
        vol = df.trgt[i]
        da = df.chiusura_vera[i:i+length]
        dmax = df.massimo_vera[i+1:i+length+1]
        dmin = df.minimo_vera[i+1:i+length+1]
        da.reset_index(drop=True, inplace = True)
        dmax.reset_index(drop=True, inplace = True)
        dmin.reset_index(drop = True, inplace = True)
        upper_band = da[0]*(1+(height_high*vol))
        lower_band = da[0]*(1-(height_low*vol))

        high_barrier.append(upper_band)
        low_barrier.append(lower_band)
        vert_barrier.append(df.data[i+length])

        #print("upper",upper_band)
        #print("upper",upper_band)
        etiquetas = []


        for j in range(length):
          if (dmax[j] >= upper_band and dmin[j] > lower_band):
            etiquetas.append(1)
          elif (dmax[j] < upper_band and dmin[j] <= lower_band):
            etiquetas.append(-1)
          else:
            etiquetas.append(0)

        boolean = [value != 0 for value in etiquetas]
        etiquetas = np.array(etiquetas)
        boolean = np.array(boolean)

        if any(boolean):
          output.append(etiquetas[boolean][0])
        else:
          output.append(0)

    for h in range(len(data)-length, len(data)):
      output.append(np.nan)

      da = df.chiusura_vera[h]
      vol = df.trgt[h]
      upper_band = da*(1+(height_high*vol))
      lower_band = da*(1-(height_low*vol))

      high_barrier.append(upper_band)
      low_barrier.append(lower_band)
      vert_barrier.append(np.nan)

    #datacopy.drop(datacopy.index[-(length+1):],axis=0, inplace=True)
    datacopy["high_barrier"] = np.array(high_barrier)
    datacopy["low_barrier"] = np.array(low_barrier)
    datacopy["vert_barrier"] = np.array(vert_barrier)
    datacopy["labels"] = np.array(output)

    count_0 = np.count_nonzero(datacopy["labels"] == 0)
    # Conta quanti 1 ci sono nell'array
    count_1 = np.count_nonzero(datacopy["labels"]== 1)
    # Conta quanti -1 ci sono nell'array
    count_minus_1 = np.count_nonzero(datacopy["labels"] == -1)
    print(f"Numero di 0: {count_0}")
    print(f"Numero di 1: {count_1}")
    print(f"Numero di -1: {count_minus_1}")
    metrica_balance = 0
    return datacopy, metrica_balance

In [3]:
def crossover(arr1,arr2):
  return(arr1>arr2)&(arr1.shift(1)<arr2.shift(1))
def crossunder(arr1,arr2):
  return(arr1<arr2)&(arr1.shift(1)>arr2.shift(1))
def drawdown(equity):#valore equity - valore max
  maxvalue=equity.expanding(0).max()
  drawdown=equity-maxvalue
  drawdown_series=pd.Series(drawdown, index=equity.index)
  return drawdown_series
def operation_number(operations):
  return operations.count()
def max_gain(operations):
  return round(operations.max(),2)
def max_drawdown_nozero(equity):
    dd=drawdown(equity)
    return round(dd[dd<0].min(),2)
def avgdrawdown_nozero(equity):#sofferenza media del sistema
    dd=drawdown(equity)
    return round(dd[dd<0].mean(),2)
def avg_delay_between_peaks(equity):#ritardo nel compiere massimi
  work_df=pd.DataFrame(equity,index=equity.index)
  work_df['drawdown']=drawdown(equity)
  work_df['delay_elements']=work_df['drawdown'].apply(lambda x:1 if x<0 else 0)
  work_df['resets']=np.where(work_df['drawdown']==0,1,0)
  work_df['cumsum']=work_df['resets'].cumsum()
  work_df.dropna(inplace=True)
  a=pd.Series(work_df['delay_elements'].groupby(work_df['cumsum']).sum())
  return round(a.mean(),2)
def delay_between_peaks(equity):#ritardo nel compiere massimi
  work_df=pd.DataFrame(equity,index=equity.index)
  work_df['drawdown']=drawdown(equity)
  work_df['delay_elements']=work_df['drawdown'].apply(lambda x:1 if x<0 else 0)
  work_df['resets']=np.where(work_df['drawdown']==0,1,0)
  work_df['cumsum']=work_df['resets'].cumsum()
  a=pd.Series(work_df['delay_elements'].groupby(work_df['cumsum']).cumsum())
  return a
def max_delay_between_peaks(equity):#quanto bisogna aspettare per vedere nuovi massimi
  a=delay_between_peaks(equity)
  return a.max()
def reward_risk_ratio(operations):
  if operations[operations<=0].mean()!=0:
    return round((operations[operations>0].mean()/-operations[operations<=0].mean()),2)
  else:
    return np.inf
def percent_win(operations):
  return round((operations[operations>0].count()/operations.count()*100),2)
def profit_factor(operations):
  a=gross_profit(operations)
  b=gross_loss(operations)
  if b!=0:
    return round(abs(a/b),2)
  else:
    return round(abs(a/0.000000001),2)
def gross_profit(operations):
  return round(operations[operations>0].sum(),2)
def gross_loss(operations):
  return round(operations[operations<0].sum(),2)
def avg_loss(operations):
  return round(operations[operations<0].mean(),2)
def max_loss(operations):
  return round(operations.min(),2)
def max_loss_date(operations):
  return operations.idxmin()
def max_win(operations):
  return round(operations.max(),2)
def max_win_date(operations):
  return operations.idxmax()
def avg_gain(operations):
  return round(operations[operations>0].mean(),2)
def avg_trade(operations):
  return round(operations.mean(),2)
def max_draw_down(equity):
  dd=drawdown(equity)
  return round(dd.min(),2)
def avgdown_nozero(equity):#sofferenza media del sistema
    dd=drawdown(equity)
    return round(dd[dd<0].mean(),2)
def profit(equity):
  return round(equity[-1],2)

In [4]:
def plot_drawdown(equity,color):
  dd=drawdown(equity)
  plt.figure(figsize=(14,8),dpi=100)
  plt.plot(dd,color=color)
  plt.fill_between(dd.index,0,dd,color=color)
  plt.xlabel="time"
  plt.ylabel="profit/Loss"
  plt.title('Draw Down')
  plt.show()
  return
def plot_equity(equity,color):
  plt.figure(figsize=(14,8),dpi=100)
  plt.plot(equity,color=color)
  plt.xlabel="time"
  plt.ylabel="profit/Loss"
  plt.title('equity line')
  plt.show()
  return
def plot_equity_heatmap(operations,annotations):
  monthly=operations.resample('M').sum()
  toHeatMap=pd.DataFrame(monthly)
  toHeatMap['Year']=toHeatMap.index.year
  toHeatMap['Month']=toHeatMap.index.month
  show=toHeatMap.groupby(by=['Year','Month']).sum().unstack()
  show.columns=['gennaio','febbraio','marzo','aprile','maggio','giugno','luglio','agosto','settembre','ottobre','novembre','dicembre']
  plt.figure(figsize=(10,7),dpi=120)
  sns.heatmap(show,cmap="RdYlGn",linecolor="white",linewidth=0.1,annot=annotations,vmin=-max(monthly.min(),monthly.max()),vmax=monthly.max())
  return
def plot_monthly_bias_histogram(operations):
  monthly=pd.DataFrame(operations.fillna(0)).resample('M').sum()
  monthly['Month']=monthly.index.month
  biasMonthly=[]
  months=[]
  for month in range(1,13):
    months.append(month)
  for month in months:
    biasMonthly.append(monthly[(monthly['Month']==month)].mean())

  biasMonthly=pd.DataFrame(biasMonthly)
  column=biasMonthly.columns[0]
  colors=pd.Series()
  colors=biasMonthly[column].apply(lambda x:'green' if x>0 else 'red')
  n_groups=len(biasMonthly)
  plt.subplots(figsize=(10,7),dpi=100)
  index=np.arange(n_groups)
  bar_width=0.35
  opacity=1
  rectls1=plt.bar(index,biasMonthly[column],bar_width,alpha=opacity,color=colors,label='yearly statistics')
  #plt.xlabel('years')
  #plt.ylabel('profit-loss')
  months_names=['gennaio','febbraio','marzo','aprile','mggio','giugno','luglio','agosto','settembre','ottobre','novembre','dicembre']
  plt.xticks(index,months_names,rotation=45)
  plt.grid(True)
  plt.show()
  return
def plot_annual_histogram(operations):
  yearly=operations.resample('A').sum()
  colors=pd.Series()
  colors=yearly.apply(lambda x:'green' if x>0 else 'red')
  n_groups=len(yearly)
  plt.subplots(figsize=(10,7),dpi=100)
  index=np.arange(n_groups)
  bar_width=0.35
  opacity=1
  rectls1=plt.bar(index,yearly,bar_width,alpha=opacity,color=colors,label='yearly statistics')

  #plt.xlabel('years')
  #plt.ylabel('profit-loss')
  plt.xticks(index,yearly.index.year,rotation=90)
  plt.grid(True)
  plt.show()
  return

In [5]:
def performance_report(trading_system,operations,closed_equity,open_equity):
  print("Performance report")
  print("")
  print("profit:                ",profit(open_equity))
  print("operations:            ",operation_number(operations))
  print("avarage Trade:         ",avg_trade(operations))
  print("")
  print("profit factor:         ",profit_factor(operations))
  print("gross profit:          ",gross_profit(operations))
  print("percent winning trade: ",percent_win(operations))
  print("reward risk ratio:     ",reward_risk_ratio(operations))
  print("")
  print("max gain:              ",max_gain(operations))
  print("avarage gain:          ",avg_gain(operations))
  print("max loss:              ",max_loss(operations))
  print("avarage loss:          ",avg_loss(operations))
  print("")
  print("avg open draw down:    ",avgdrawdown_nozero(open_equity))
  print("max open draw down:    ",max_drawdown_nozero(open_equity))
  print("")
  print("avg closed draw down:   ",avgdrawdown_nozero(trading_system.closed_equity))
  print("max closed draw down:   ",max_drawdown_nozero(trading_system.closed_equity))
  print("")
  print("avg delay between peaks:",avgdrawdown_nozero(trading_system.closed_equity))
  print("max delay between peaks:",max_drawdown_nozero(trading_system.closed_equity))
  plot_equity(trading_system.open_equity,"green")
  plot_drawdown(trading_system.open_equity,"red")
  plot_annual_histogram(operations)
  plot_monthly_bias_histogram(operations)
  plot_equity_heatmap(operations,False)
  return

In [6]:
def marketposition_generator(enter_rules,exit_rules):
  service_df=pd.DataFrame(index=enter_rules.index)
  service_df['enter_rules']=enter_rules
  service_df['exit_rules']=exit_rules
  status=0
  mp=[]
  #print(enter_rules)
  #print(exit_rules)
  for (i,j) in zip(enter_rules,exit_rules):#lega enter e exit con il loro valore temporale
    #if status==0:
    if i ==1:
      status+=1
    else:
      if j==-1 and status>=1:
        status-=1
    mp.append(status)
  service_df['mp_new']=mp
  service_df.mp_new=service_df.mp_new.shift(1)#ritardo mp
  service_df.iloc[0,2]=0
  service_df.to_csv("marketposition_generator.csv")
  print(service_df[0:50])
  return service_df.mp_new
import math
def tick_correction_up(level,tick):
  if level != level:
    level=0
  multiplier=math.ceil(level/tick)
  return multiplier*tick
def tick_correction_down(level,tick):
  if level != level:
    level=0
  multiplier=math.floor(level/tick)
  return multiplier*tick
def stop_check(dataframe,rules,level,direction):
  service_dataframe=pd.DataFrame()
  service_dataframe['rules']=rules
  service_dataframe['level']=level #livelli di ingresso boolenani
  service_dataframe['low']=dataframe.Low
  service_dataframe['high']=dataframe.High

  if direction=='long':
    service_dataframe['new_rules']=np.where((service_dataframe.rules==True)&(service_dataframe.high.shift(-1)>=  service_dataframe.level.shift(-1)),True,False)
  if direction=='short':
    service_dataframe['new_rules']=np.where((service_dataframe.rules==True)&(service_dataframe.low.shift(-1)<=  service_dataframe.level.shift(-1)),True,False)
  return(service_dataframe.new_rules)

def limit_check(dataframe,rules,level,direction):
  service_dataframe=pd.DataFrame()
  service_dataframe['rules']=rules
  service_dataframe['level']=level
  service_dataframe['low']=dataframe.Low
  service_dataframe['high']=dataframe.High
  if direction=='long':
    service_dataframe['new_rules']=np.where((service_dataframe.rules==True)&(service_dataframe.low.shift(-1)<=  service_dataframe.level.shift(-1)),True,False)
  if direction=='short':
    service_dataframe['new_rules']=np.where((service_dataframe.rules==True)&(service_dataframe.high.shift(-1)>=  service_dataframe.level.shift(-1)),True,False)
  return(service_dataframe.new_rules)


In [7]:
def apply_trading_system(imported_dataframe,bigpoint_value,tick,direction,order_type,enter_level, enter_rules,exit_rules):
  dataframe=imported_dataframe.copy()
  if order_type=="stop":
    enter_rules=stop_check(dataframe,enter_rules,enter_level,direction)
  if order_type=="limit":
    enter_rules=limit_check(dataframe,enter_rules,enter_level,direction)
  dataframe['enter_level']=enter_level
  dataframe['enter_rules']=enter_rules.apply(lambda x: 1 if x==True else 0)
  dataframe['exit_rules']=exit_rules.apply(lambda x: -1 if x==True else 0)

  dataframe['mp']=marketposition_generator(dataframe.enter_rules,dataframe.exit_rules)

  if order_type=="market":
    dataframe['entry_price']=np.where((dataframe.mp.shift(1)==0)&(dataframe.mp>=1),dataframe.Open, np.nan)
    if instruments==1:
      dataframe['number_of_stocks']=np.where((dataframe.mp.shift(1)==0)&(dataframe.mp>=1),operation_money/dataframe.Open,np.nan)


  if order_type=="stop":
    if direction=="long":
      dataframe.enter_level=dataframe.enter_level.apply(lambda x: tick_correction_up(x,tick))
      real_entry=np.where(dataframe.Open>dataframe.enter_level,dataframe.Open,dataframe.enter_level)
      dataframe['entry_price']=np.where((dataframe.mp.shift(1)==0)&(dataframe.mp>=1),real_entry,np.nan)
    if direction=="short":
      dataframe.enter_level=dataframe.enter_level.apply(lambda x: tick_correction_down(x,tick))
      real_entry=np.where(dataframe.Open<dataframe.enter_level,dataframe.Open,dataframe.enter_level)
      dataframe['entry_price']=np.where((dataframe.mp.shift(1)==0)&(dataframe.mp>=1),real_entry,np.nan)
    if instruments==1:
      dataframe['number_of_stocks']=np.where((dataframe.mp.shift(1)==0)&(dataframe.mp>=1),operation_money/real_entry,np.nan)
    #if instruments==1:
      #dataframe['number_of_stocks']=dataframe['number_of_stocks'].apply(lambda x: round(x,0)).fillna(method='ffill')


  if order_type=="limit":
    if direction=="long":
      dataframe.enter_level=dataframe.enter_level.apply(lambda x: tick_correction_down(x,tick))
      real_entry=np.where(dataframe.Open<dataframe.enter_level,dataframe.Open,dataframe.enter_level)
      dataframe['entry_price']=np.where((dataframe.mp.shift(1)==0)&(dataframe.mp>=1),real_entry,np.nan)

    if direction=="short":
      dataframe.enter_level=dataframe.enter_level.apply(lambda x: tick_correction_up(x,tick))
      real_entry=np.where(dataframe.Open>dataframe.enter_level,dataframe.Open,dataframe.enter_level)
      dataframe['entry_price']=np.where((dataframe.mp.shift(1)==0)&(dataframe.mp>=1),real_entry,np.nan)

    if instruments==1:
      dataframe['number_of_stocks']=np.where((dataframe.mp.shift(1)==0)&(dataframe.mp>=1),operation_money/real_entry,np.nan)

  dataframe['entry_price']=dataframe['entry_price'].fillna(method='ffill')
  dataframe['events_in']=np.where((dataframe.mp>=1)&(dataframe.mp.shift(1)==0),'entry','')
  if instruments==1:
    dataframe['number_of_stocks']=dataframe['number_of_stocks'].apply(lambda x: round(x,0)).fillna(method='ffill')




  if direction=="long":
    if instruments==1:
      dataframe['open_operations']=(dataframe.Close-dataframe.entry_price)*dataframe.number_of_stocks
      dataframe['open_operations']=np.where((dataframe.mp==0)&(dataframe.mp.shift(-1)==0),(dataframe.Open.shift(-1)-dataframe.entry_price)*dataframe.number_of_stocks-2*costs,dataframe.open_operations)
    if instruments==2:
      dataframe['open_operations']=(-dataframe.entry_price+dataframe.Close)*bigpoint_value
      dataframe['open_operations']=np.where((dataframe.mp>=1)&(dataframe.mp.shift(-1)==0),(dataframe.Open.shift(-1)-dataframe.entry_price)*bigpoint_value-2*costs,dataframe.open_operations)

  else:
    if instruments==1:
      dataframe['open_operations']=(dataframe.entry_price-dataframe.Close)*dataframe.number_of_stocks
      dataframe['open_operations']=np.where((dataframe.mp>=1)&(dataframe.mp.shift(-1)==0),(-dataframe.Open.shift(-1)+dataframe.entry_price)*dataframe.number_of_stocks-2*costs,dataframe.open_operations)
    if instruments==2:
      dataframe['open_operations']=(dataframe.entry_price-dataframe.Close)*bigpoint_value
      dataframe['open_operations']=np.where((dataframe.mp>=1)&(dataframe.mp.shift(-1)==0),(-dataframe.Open.shift(-1)+dataframe.entry_price)*bigpoint_value-2*costs,dataframe.open_operations)


  dataframe['open_operations']=np.where(dataframe.mp>=1,dataframe.open_operations,0)
  dataframe['events_out']=np.where((dataframe.mp>=1)&(dataframe.exit_rules==-1),'exit','')
  dataframe['operations']=np.where((dataframe.exit_rules==-1)&(dataframe.mp>=1),dataframe.open_operations,np.nan)
  dataframe['closed_equity']=dataframe.operations.fillna(0).cumsum()
  dataframe['open_equity']=dataframe.closed_equity+dataframe.open_operations-dataframe.operations.fillna(0)
  dataframe.to_csv('trading_system_export.csv')
  return dataframe

In [8]:
def costs_adder(options,costs):
  new_operations=operations.apply(lambda x:x-2*costs)
  return new_operations
def noise_adder(operations,percentage_noise_addiction):
  new_operations=[]
  for el in operations:
    factor=np.random.uniform(-1,1)
    new_operations.append(round(el+factor*(percentage_noise_addiction/100)*abs(el),2))
    new_operations_series=pd.Series(new_operations,index=operations.index,name='operations')
    return new_operations_series

In [9]:
import datetime
def evolved_montecarlo_analysis(Costs,PercentageNoiseAddiction,OperationsPercentage,NumberOfShuffles):
  original_operations=operations
  if Costs!=0:
    original_operations=costs_adder(operations,Costs)
  if PercentageNoiseAddiction!=0:
    original_operations=noise_adder(operations,PercentageNoiseAddiction)
  original_equity=original_operations.cumsum()
  original_profit=round(original_operations.sum(),2)
  original_drawdown=drawdown(original_operations)
  original_max_drawdown=round(original_drawdown.min(),2)
  if OperationsPercentage==100:
    matrix_of_equities=pd.DataFrame(original_equity).reset_index()
    matrix_of_equities.drop(matrix_of_equities.columns[0],axis=1,inplace=True)
    matrix_of_equities.columns=['original']

    matrix_of_drawdowns=pd.DataFrame(original_drawdown).reset_index()
    matrix_of_drawdowns.drop(matrix_of_drawdowns.columns[0],axis=1,inplace=True)
    matrix_of_drawdowns.columns=['original']

  else:
    cutnumber=int(len(operations)*int(OperationsPercentage)/100)
    matrix_of_equities=pd.DataFrame(original_equity[:cutnumber]).reset_index()
    matrix_of_equities.drop(matrix_of_equities.columns[0],axis=1,inplace=True)
    matrix_of_equities.columns=['original']

    matrix_of_drawdowns=pd.DataFrame(original_drawdown[:cutnumber]).reset_index()
    matrix_of_drawdowns.drop(matrix_of_drawdowns.columns[0],axis=1,inplace=True)
    matrix_of_drawdowns.columns=['original']

  max_drawdown_list=[]
  fraction=OperationsPercentage/100
  i=0
  start=datetime.datetime.now()
  while i<NumberOfShuffles:
    my_permutation=original_operations.sample(frac=fraction).reset_index(drop=True)
    my_permutation=pd.Series(my_permutation)
    new_equity=my_permutation.cumsum()
    new_drawdown=drawdown(new_equity)
    matrix_of_equities['shuffle'+str(i+1)]=new_equity
    matrix_of_drawdowns['shuffle'+str(i+1)]=new_drawdown
    max_drawdown_list.append(new_drawdown.min())
    i+=1
  end=datetime.datetime.now()
  timespent=end-start
  print("Shuffles executed in:",timespent)
  worst_drawdown=round(matrix_of_drawdowns.min().min(),2)
  worst_drawdown_index=matrix_of_drawdowns.min().idxmin()
  worst_profit=round(matrix_of_equities[worst_drawdown_index][matrix_of_equities[worst_drawdown_index].count()-1],2)

  best_drawdown=round(matrix_of_drawdowns.min().max(),2)
  best_drawdown_index=matrix_of_drawdowns.min().idxmax()
  best_profit=round(matrix_of_equities[best_drawdown_index][matrix_of_equities[best_drawdown_index].count()-1],2)

  print("oringinal Profit/Loss:",original_profit)
  print("original_max_draw_down",original_max_drawdown)
  print("worst equity index:",worst_drawdown_index)
  print("worst equity profit/loss:",worst_profit)
  print("worst equity max draw down:",worst_drawdown)
  print("best equity index:",best_drawdown_index)
  print("best equity profit/loss:",best_profit)
  print("best equity max draw down:",best_drawdown)
  MaxDrawDown95=round(np.percentile(max_drawdown_list,5),2)
  riskfactor95=round(np.percentile(max_drawdown_list,5)/original_max_drawdown,2)
  riskfactor=round(min(max_drawdown_list)/original_max_drawdown,2)
  print('95 percentile Montecarlo Max drown down',MaxDrawDown95)
  print('95 percentile Montecarlo risk factor',riskfactor95)
  print("worst montecarlo max draw down:",round(min(max_drawdown_list),2))
  print("montecarlo risk factor on max draw down:",riskfactor)
  plt.subplots(figsize=(14,8),dpi=300)
  plt.plot(matrix_of_equities.iloc[:,1:],color='grey',linewidth=2,linestyle="-")
  plt.plot(matrix_of_equities[worst_drawdown_index],color='red',linewidth=2,linestyle="-",label='worst equity line')
  plt.plot(matrix_of_equities[best_drawdown_index],color='green',linewidth=2,linestyle="-",label='best equity line')

  if OperationsPercentage==100:
    plt.plot(original_equity.reset_index(drop=True),color='blue',linewidth=2,linestyle="-",label='original equity line')
  else:
    plt.plot(original_equity.reset_index(drop=True)[:cutnumber],color='blue',linewidth=2,linestyle="-")
  plt.grid(True)
  plt.legend()
  plt.show()
  output=[]
  output=sorted(max_drawdown_list)
  n_groups=NumberOfShuffles
  fig,ax=plt.subplots(figsize=(14,8),dpi=300)
  index=np.arange(n_groups)
  bar_width=0.5
  opacity=1
  error_config={'ecolor':'0.3'}
  rectls1=ax.bar(index,output,bar_width,alpha=opacity,color='red',error_kw=error_config,label='Drawdowns distribution')
  ax.set_xlabel=('shuffles number')
  ax.set_ylabel=('montecarlo drawdown')
  ax.grid(True)
  plt.show()
  percentiles=np.zeros(100)
  i=0
  while i<100:
      percentiles[i]=np.percentile(max_drawdown_list,int(i))
      i+=1
  plt.figure(figsize=(14,8),dpi=300)
  #plt.xlabel('percentile values')
  #plt.ylabel('monetary drawdow')
  plt.axhline(y=int(original_max_drawdown),xmin=0,xmax=100,linewidth=2,color='green')
  plt.axhline(y=int(MaxDrawDown95),xmin=0,xmax=100,linewidth=2,color='blue')
  plt.plot(percentiles,color='red',linewidth=2,linestyle="-")
  plt.xticks(np.arange(0,99,5))
  plt.grid(True)
  plt.legend()
  plt.show()
  return

In [10]:
df=pd.read_csv('/content/drive/MyDrive/FINANZA/dati ogni minuto dal 97 a oggi/dati_trattati.csv')
#df=df.drop('labels',axis=1)

df,_=triple_barrier_m(df,5,2.105,2.135)#.shift(1)
#print(df)
df['data']= pd.to_datetime(df['data'])
df['data2']= pd.to_datetime(df['data'])
df = df.set_index('data')
df['day']=df.index.day
df['month']=df.index.month
df['year']=df.index.year
df['hour']=df.index.hour
df['minute']=df.index.minute
df['dayofyear']=df.index.dayofyear
df['dayofweek']=df.index.dayofweek
df['quarter']=df.index.quarter#quadrimestre in corso
df['Close']=df.chiusura_vera
df['High']=df.massimo_vera
df['Low']=df.minimo_vera
df['Open']=df.apertura_vera
df['labels']=df['labels']+1

#print(df.labels)
df

Numero di 0: 12373
Numero di 1: 12363
Numero di -1: 12368


,apertura,massimo,minimo,chiusura,chiusura_vera,apertura_vera,massimo_vera,minimo_vera,media50,media20,...,year,hour,minute,dayofyear,dayofweek,quarter,Close,High,Low,Open
data,,,,,,,,,,,,,,,,,,,,,
2006-07-21 19:41:00,58.451877,53.868429,59.865164,58.231484,1199.75,1202.00,1204.75,1198.25,1216.345,1205.7750,...,2006,19,41,202,4,3,1199.75,1204.75,1198.25,1202.00
2006-07-24 00:02:00,58.657207,64.339760,60.758445,73.505561,1213.25,1200.00,1213.25,1198.00,1215.895,1206.6250,...,2006,0,2,205,0,3,1213.25,1213.25,1198.00,1200.00
2006-07-24 16:27:00,73.008544,68.859781,75.438185,76.138397,1220.50,1212.75,1220.50,1212.00,1215.585,1207.7625,...,2006,16,27,205,0,3,1220.50,1220.50,1212.00,1212.75
2006-07-24 17:51:00,75.378028,66.822322,73.754270,73.616415,1222.00,1219.50,1222.00,1215.50,1215.300,1209.1375,...,2006,17,51,205,0,3,1222.00,1222.00,1215.50,1219.50
2006-07-24 21:13:00,73.611519,66.364150,74.224575,68.641555,1219.00,1221.50,1223.25,1218.75,1214.815,1210.1750,...,2006,21,13,205,0,3,1219.00,1223.25,1218.75,1221.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-20 16:32:00,255.541613,235.940298,257.606611,262.707124,4578.50,4575.75,4578.50,4571.25,4575.415,4594.9500,...,2023,16,32,201,3,3,4578.50,4578.50,4571.25,4575.75
2023-07-20 16:53:00,264.204042,245.033691,266.771407,262.113564,4577.75,4580.25,4583.50,4577.75,4576.225,4593.6750,...,2023,16,53,201,3,3,4577.75,4583.50,4577.75,4580.25
2023-07-20 17:24:00,261.531114,246.850664,264.939238,270.783153,4585.75,4578.00,4586.00,4577.50,4577.155,4592.5750,...,2023,17,24,201,3,3,4585.75,4586.00,4577.50,4578.00


In [11]:
print(df.columns)
colonne=['chiusura_vera','apertura_vera', 'massimo_vera', 'minimo_vera','high_barrier', 'low_barrier', 'vert_barrier', 'labels']
df[colonne]

Index(['apertura', 'massimo', 'minimo', 'chiusura', 'chiusura_vera',
       'apertura_vera', 'massimo_vera', 'minimo_vera', 'media50', 'media20',
       'media2', 'log', 'trgt', 'expmedia2', 'expmedia20', 'expmedia50',
       'Hurst100', 'high_barrier', 'low_barrier', 'vert_barrier', 'labels',
       'data2', 'day', 'month', 'year', 'hour', 'minute', 'dayofyear',
       'dayofweek', 'quarter', 'Close', 'High', 'Low', 'Open'],
      dtype='object')


,chiusura_vera,apertura_vera,massimo_vera,minimo_vera,high_barrier,low_barrier,vert_barrier,labels
data,,,,,,,,
2006-07-21 19:41:00,1199.75,1202.00,1204.75,1198.25,1205.900988,1193.685442,2006-07-25 15:42:00,2.0
2006-07-24 00:02:00,1213.25,1200.00,1213.25,1198.00,1225.457243,1201.214288,2006-07-25 16:40:00,2.0
2006-07-24 16:27:00,1220.50,1212.75,1220.50,1212.00,1233.674142,1207.510975,2006-07-25 20:03:00,1.0
2006-07-24 17:51:00,1222.00,1219.50,1222.00,1215.50,1235.230641,1208.955270,2006-07-25 21:39:00,1.0
2006-07-24 21:13:00,1219.00,1221.50,1223.25,1218.75,1232.359856,1205.827871,2006-07-26 15:56:00,2.0
...,...,...,...,...,...,...,...,...
2023-07-20 16:32:00,4578.50,4575.75,4578.50,4571.25,4593.124578,4564.080919,nan,NaN
2023-07-20 16:53:00,4577.75,4580.25,4583.50,4577.75,4589.481204,4566.183637,nan,NaN
2023-07-20 17:24:00,4585.75,4578.00,4586.00,4577.50,4599.462708,4572.229977,nan,NaN


In [ ]:
df['date2'] = pd.to_datetime(df['data2'])

# Inizializza una colonna 'take_profit' con tutti False
df['take_profit'] = False

# Itera attraverso le righe e imposta 'take_profit' a True se il take profit o lo stop loss viene toccato, altrimenti imposta True dopo 5 candele
for date, row in df.iterrows():
    future_dates = df.loc[df.index > date].head(5).index
    condition1 = (df.loc[df.index.isin(future_dates)]['massimo_vera'] >= row['high_barrier']).any()
    condition2 = (df.loc[df.index.isin(future_dates)]['minimo_vera'] <= row['low_barrier']).any()
    #print("data",date)
    if condition1:
        future_date1 = df.loc[df.index.isin(future_dates)].index[0]
        #print("future data",future_date)
    if condition2:
        future_date2 = df.loc[df.index.isin(future_dates)].index[0]
        #print("future data",future_date)

    if condition1 and condition2:
        chosen_date = min(future_date1, future_date2)
    elif future_date1:
        chosen_date = future_date1
    elif future_date2:
        chosen_date = future_date2
    if condition1 or condition2:
    #   future_date = future_rows.loc[(condition1 | condition2)].index[0]  # Ottieni la data futura dalla prima riga che soddisfa la condizione
    #    #df.at[future_date, 'take_profit'] = Truela data futura dalla prima riga che soddisfa la condizione
        df.at[chosen_date, 'take_profit'] = True
    else:
        df.at[future_dates[-1], 'take_profit'] = True  # Imposta True dopo 5 date successive se nessuna condizione viene soddisfatta

df

IndexError: ignored

In [ ]:
df

In [ ]:
print(df.columns)
colonne=['chiusura_vera','apertura_vera', 'massimo_vera', 'minimo_vera','high_barrier', 'low_barrier', 'vert_barrier', 'labels','take_profit']
df[colonne]

In [ ]:
bigpointvalue=1
df.dropna(inplace=True)
costs=0#costi fissi per trading
instruments=2 #1=azioni/valute, 2=future
operation_money=1000#investimento monetario
tick=0.000
direction="long"#oppure short
order_type="market"#oppure limit o stop

enter_rules=(df.labels==2)    #|(df.hour>12)
enter_level=(df.Close)      #df.hhv11.shift(1)#*60
exit_rules= df['take_profit']#(df.High>=df.high_barrier.shift(-5))|(df.index>=df.vert_barrier.shift(-5))#|(df.Low<=df.low_barrier)#|(df.index>=df.vert_barrier.shift(-5))
print(exit_rules[0:50])
trading_system=apply_trading_system(df,bigpointvalue,tick,direction,order_type,enter_level,enter_rules,exit_rules)
operations=trading_system.operations.dropna()
if operations.count()>0:
  performance_report(trading_system,operations,trading_system.closed_equity,trading_system.open_equity)
else:
  print("Non ho fatto nessuna operazione capo!")

In [ ]:
costs=0
PercentageNoiseAddiction=0
OperationsPercentage=100
NumberOfShuffles=1000
evolved_montecarlo_analysis(costs,PercentageNoiseAddiction,OperationsPercentage,NumberOfShuffles)

In [ ]:
def equity_control_wp(operations,period,treshold):
  df=pd.DataFrame(operations)
  df.columns=['original']
  df['window_profit']=df.original.rolling(period).sum()
  df['controlled']=np.where(df.window_profit.shift(1)>treshold,df.original,0)
  df['equity_original']=df.original.cumsum()
  df['equity_controlled']=df.controlled.cumsum()
  return df
period=10
treshold=500
df=equity_control_wp(operations,period,treshold)
service=pd.DataFrame({'original':df.equity_original,'controlled':df.equity_controlled})
plt.plot(service)
plt.show()
service2=pd.DataFrame({'original':drawdown(df.equity_original),'controlled':drawdown(df.equity_controlled)})
plt.plot(service2)
plt.show()
